# Preprocesamiento

Se importan las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Se carga el csv en un dataframe

In [2]:
data = pd.read_csv('../bd/data2.csv', dtype={'rut': str})
data

,rut,sexo,rangoEdad,plan,centroFav,tipoFav,visitas2018,visitas2019,visitas2020,visitas2021,visitas2022
0,25274441,Femenino,18 AÑOS O MENOS,COLMENA,CMD San Bernardo,Dental,0,0,0,1,0
1,25274447,Femenino,31 AÑOS - 40 AÑOS,CONSALUD,CM Parque Arauco,Médica,0,1,0,0,0
2,25274449,Femenino,31 AÑOS - 40 AÑOS,FONASA,Clínica Rancagua,Médica,0,1,0,0,0
3,25274450,Femenino,18 AÑOS O MENOS,CONSALUD,Clínica Elqui,Urgencia,0,0,0,0,0
4,25274458,Femenino,18 AÑOS O MENOS,BANMEDICA,Clínica Santiago,Urgencia,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3187257,18453648,Femenino,19 AÑOS - 30 AÑOS,FONASA,Telemedicina,Telemedicina,0,0,0,0,0
3187258,18453651,Femenino,19 AÑOS - 30 AÑOS,FONASA,Clínica Rancagua,Imagenologia,0,0,0,1,0
3187259,18453652,Masculino,19 AÑOS - 30 AÑOS,FONASA,CMD San Miguel - Gran Avenida,Médica,0,0,0,0,1
3187260,1845366,Masculino,19 AÑOS - 30 AÑOS,FONASA,CMD La Florida,Médica,0,0,0,0,1


Se elimina el rut, ya que no será utilizado para la regresión

In [3]:
data_noRut = data.drop(['rut'], axis=1)
data_noRut

,sexo,rangoEdad,plan,centroFav,tipoFav,visitas2018,visitas2019,visitas2020,visitas2021,visitas2022
0,Femenino,18 AÑOS O MENOS,COLMENA,CMD San Bernardo,Dental,0,0,0,1,0
1,Femenino,31 AÑOS - 40 AÑOS,CONSALUD,CM Parque Arauco,Médica,0,1,0,0,0
2,Femenino,31 AÑOS - 40 AÑOS,FONASA,Clínica Rancagua,Médica,0,1,0,0,0
3,Femenino,18 AÑOS O MENOS,CONSALUD,Clínica Elqui,Urgencia,0,0,0,0,0
4,Femenino,18 AÑOS O MENOS,BANMEDICA,Clínica Santiago,Urgencia,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3187257,Femenino,19 AÑOS - 30 AÑOS,FONASA,Telemedicina,Telemedicina,0,0,0,0,0
3187258,Femenino,19 AÑOS - 30 AÑOS,FONASA,Clínica Rancagua,Imagenologia,0,0,0,1,0
3187259,Masculino,19 AÑOS - 30 AÑOS,FONASA,CMD San Miguel - Gran Avenida,Médica,0,0,0,0,1
3187260,Masculino,19 AÑOS - 30 AÑOS,FONASA,CMD La Florida,Médica,0,0,0,0,1


Se crean los codificadores de los datos categóricos

In [4]:
le_rangoEdad = LabelEncoder()
le_sexo = LabelEncoder()
le_plan = LabelEncoder()
le_centro = LabelEncoder()
le_tipo = LabelEncoder()

Se codifican las variables

In [5]:
rangoEdad_encoded = le_rangoEdad.fit_transform(data_noRut['rangoEdad'])
sexo_encoded = le_sexo.fit_transform(data_noRut['sexo'])
plan_encoded = le_plan.fit_transform(data_noRut['plan'])
centro_encoded = le_centro.fit_transform(data_noRut['centroFav'])
tipo_encoded = le_tipo.fit_transform(data_noRut['tipoFav'])

encoded_df = data_noRut[['visitas2018','visitas2019','visitas2020','visitas2021','visitas2022']]
encoded_df.insert(0, column='sexo_encoded', value=sexo_encoded)
encoded_df.insert(1, column='rangoEdad_encoded', value=rangoEdad_encoded)
encoded_df.insert(2, column='plan_encoded', value=plan_encoded)
encoded_df.insert(3, column='centro_encoded', value=centro_encoded)
encoded_df.insert(4, column='tipo_encoded', value=tipo_encoded)

encoded_df

,sexo_encoded,rangoEdad_encoded,plan_encoded,centro_encoded,tipo_encoded,visitas2018,visitas2019,visitas2020,visitas2021,visitas2022
0,0,0,1,44,0,0,0,0,1,0
1,0,2,2,26,7,0,1,0,0,0
2,0,2,4,54,7,0,1,0,0,0
3,0,0,2,50,12,0,0,0,0,0
4,0,0,0,55,12,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3187257,0,1,4,59,11,0,0,0,0,0
3187258,0,1,4,54,4,0,0,0,1,0
3187259,1,1,4,45,7,0,0,0,0,1
3187260,1,1,4,34,7,0,0,0,0,1


Se separan los datos en un set de entrenamiento (80%) y un set de pruebas (20%)

In [6]:
features = encoded_df.drop(['visitas2022'], axis=1)
labels = encoded_df[['visitas2022']]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Regresión Lineal

Se crea el modelo de regresión lineal

In [7]:
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

Se crean las predicciones

In [8]:
y_pred = reg.predict(X_test)
y_pred

array([[ 0.66127367],
       [ 0.05906948],
       [ 0.93281196],
       ...,
       [ 0.00864973],
       [-0.05219495],
       [ 0.42921298]])

Se muestran los indicadores del modelo

In [9]:
MSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
R2 = r2_score(y_test,y_pred)
Score = reg.score(X_test,y_test)
print('MSE: ',MSE)
print('Score: ',Score)
print('R2_Score: ',R2)

MSE:  0.33084812073297365
Score:  0.47830871131537334
R2_Score:  0.47830871131537334


# Postprocesamiento

Se redondea la predicción

In [10]:
y_pred = np.round(y_pred).astype(int)

Se agrega la predicción a un dataframe para facilitar su visualización

In [11]:
pred_data = pd.DataFrame(y_pred, columns=['pred_visitas2022'])
pred_data

,pred_visitas2022
0,1
1,0
2,1
3,1
4,0
...,...
637448,0
637449,1
637450,0
637451,0


Se unen los datos para entender de mejor manera los datos reales vs la predicción

In [12]:
final_data = X_test
final_data = final_data.assign(real_visitas2022=y_test)
final_data.reset_index(drop=True,inplace=True)

inverse_sexo = le_sexo.inverse_transform(final_data['sexo_encoded'])
final_data.insert(0, column='sexo', value=inverse_sexo)
final_data.drop('sexo_encoded',axis=1,inplace=True)

inverse_rangoEdad = le_rangoEdad.inverse_transform(final_data['rangoEdad_encoded'])
final_data.insert(1, column='rangoEdad', value=inverse_rangoEdad)
final_data.drop('rangoEdad_encoded',axis=1,inplace=True)

inverse_plan = le_plan.inverse_transform(final_data['plan_encoded'])
final_data.insert(2, column='plan', value=inverse_plan)
final_data.drop('plan_encoded',axis=1,inplace=True)

inverse_centro = le_centro.inverse_transform(final_data['centro_encoded'])
final_data.insert(3, column='centro', value=inverse_centro)
final_data.drop('centro_encoded',axis=1,inplace=True)

inverse_tipo = le_tipo.inverse_transform(final_data['tipo_encoded'])
final_data.insert(4, column='tipo', value=inverse_tipo)
final_data.drop('tipo_encoded',axis=1,inplace=True)

final_df = pd.concat([final_data, pred_data],axis=1)
final_df.dropna(inplace=True)
final_df.reset_index(drop=True)
final_df[['visitas2018','visitas2019','visitas2020','visitas2021','real_visitas2022','pred_visitas2022']] = final_df[['visitas2018','visitas2019','visitas2020','visitas2021','real_visitas2022','pred_visitas2022']].astype(int)
final_df

,sexo,rangoEdad,plan,centro,tipo,visitas2018,visitas2019,visitas2020,visitas2021,real_visitas2022,pred_visitas2022
0,Masculino,MAYOR O IGUAL A 75 AÑOS,BANMEDICA,CM Chillan,Laboratorio,0,0,0,0,0,1
1,Masculino,41 AÑOS - 50 AÑOS,CONSALUD,CMD La Florida,Imagenologia,0,0,1,0,0,0
2,Masculino,41 AÑOS - 50 AÑOS,PARTICULAR,CD Plaza Puente Alto,Dental,0,0,0,0,1,1
3,Femenino,19 AÑOS - 30 AÑOS,BANMEDICA,Clínica Elqui,Laboratorio,0,0,0,0,0,1
4,Femenino,MAYOR O IGUAL A 75 AÑOS,FONASA,CMD Pedro de Valdivia - Providencia,Médica,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
637448,Femenino,31 AÑOS - 40 AÑOS,COLMENA,Clínica Providencia,Médica,0,0,1,0,0,0
637449,Masculino,41 AÑOS - 50 AÑOS,FONASA,CMD German Riesco - Rancagua,Médica,0,0,0,0,1,1
637450,Femenino,51 AÑOS - 60 AÑOS,FONASA,CMD Valdivia,Imagenologia,0,0,0,1,0,0
637451,Masculino,MAYOR O IGUAL A 75 AÑOS,FONASA,CMD San Miguel - Gran Avenida,Médica,0,1,0,0,0,0


Se muestra la suma y diferencia entre visitas reales del 2022 vs predecidas

In [13]:
real = int(final_df['real_visitas2022'].sum())
pred = int(final_df['pred_visitas2022'].sum())
total = real + pred
diff = real - pred

aciertos = pd.DataFrame()
aciertos['aciertos'] = np.where((final_df['real_visitas2022'] == final_df['pred_visitas2022']),1,0)

print('Suma de las visitas reales del 2022: ', real)
print('Suma de las visitas predichas del 2022: ', pred)
print('Diferencia entre visitas reales vs predichas: ', diff)
print('Total de las visitas 2022: ', total)
#Nota: Que el total de visitas sea menor al total de filas de la base de datos está bien, ya que no está sumando los 0s.
print('-------------')
print('Aciertos de la predicción: ', aciertos['aciertos'].sum())
print('Fallos de la predicción: ', (aciertos['aciertos']==0).sum())
print('Porcentaje de aciertos: ', ((aciertos['aciertos'].sum())*100)/len(final_df))
print('Porcentaje de fallos: ', (((aciertos['aciertos']==0).sum())*100)/len(final_df))

Suma de las visitas reales del 2022:  190067
Suma de las visitas predichas del 2022:  259697
Diferencia entre visitas reales vs predichas:  -69630
Total de las visitas 2022:  449764
-------------
Aciertos de la predicción:  537546
Fallos de la predicción:  99907
Porcentaje de aciertos:  84.32715823754849
Porcentaje de fallos:  15.672841762451506


Se crea una visualización de diferencia entre las visitas reales del 2022 vs las predecidas

In [14]:
final_df['diff_visitas2022'] = final_df['real_visitas2022'] - final_df['pred_visitas2022']
dif = final_df[final_df.diff_visitas2022 >= 1]
dif.head(50)

,sexo,rangoEdad,plan,centro,tipo,visitas2018,visitas2019,visitas2020,visitas2021,real_visitas2022,pred_visitas2022,diff_visitas2022
28,Masculino,18 AÑOS O MENOS,CONSALUD,Clínica Elqui,Procedimientos,0,0,0,0,1,0,1
116,Femenino,61 AÑOS - 74 AÑOS,FONASA,Clínica Valparaíso,Procedimientos,0,0,0,0,1,0,1
120,Femenino,19 AÑOS - 30 AÑOS,FONASA,Clínica Vitacura,Médica,0,0,0,0,1,0,1
163,Femenino,MAYOR O IGUAL A 75 AÑOS,FONASA,Clínica Providencia,Procedimientos,0,0,0,0,1,0,1
194,Masculino,31 AÑOS - 40 AÑOS,BANMEDICA,Clínica Vitacura,Imagenologia,0,0,0,0,1,0,1
222,Masculino,61 AÑOS - 74 AÑOS,FONASA,Clínica Santiago,Procedimientos,0,0,0,0,1,0,1
234,Femenino,18 AÑOS O MENOS,CONSALUD,Clínica Vitacura,Médica,0,0,0,0,1,0,1
434,Masculino,19 AÑOS - 30 AÑOS,FONASA,Clínica Vitacura,Médica,0,0,0,0,1,0,1
536,Masculino,51 AÑOS - 60 AÑOS,FONASA,Clínica Valparaíso,Médica,0,0,0,0,1,0,1
623,Femenino,61 AÑOS - 74 AÑOS,FONASA,Clínica Providencia,Procedimientos,0,0,0,0,1,0,1
